# 深度学习方法

In [668]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

In [646]:
os.listdir('.')

['result',
 'predict.py',
 '.gitignore',
 'deep.ipynb',
 '.git',
 'notebook.ipynb',
 'dataset',
 '.idea']

In [647]:
baseinfo = pd.read_csv('dataset/1baseinfo.csv')

train_labels = pd.read_csv('dataset/train.csv')

test_data = pd.read_csv('dataset/evaluation_public.csv')

invest_data = pd.read_csv('dataset/4investinfo.csv')

invest_data['Sharehold'] = invest_data['Sharehold'].fillna(0).astype('float32')


150095


,EID,CreateYear,TradeType,RegisteredCapital,Type,Feature1,Feature2,Feature3,Feature4,Feature5,counts,Y,EndDate
0,732046267,1980,51,480,13,25.0,1.0,98041.0,17.0,0.0,3,0,197012
1,1338267101,2005,74,240,5,0.0,2.0,1500.0,0.0,0.0,0,0,197012
2,1057042610,2011,75,3000,6,4.0,8.0,10500.0,13.0,0.0,3,0,197012
3,400121247,1982,55,1000,6,9.0,2.0,619318.0,31.0,0.0,6,0,197012
4,970179772,1986,53,150,17,4.0,1.0,13000.0,31.0,0.0,1,1,201612


In [648]:

invest_data = invest_data[['EID']].groupby(['EID']).size().reset_index(name='counts')
invest_data['counts'] = invest_data['counts'].fillna(0).astype('int')

invest_data.head()

150095


,EID,CreateYear,TradeType,RegisteredCapital,Type,Feature1,Feature2,Feature3,Feature4,Feature5,counts,Y,EndDate
0,732046267,1980,51,480,13,25.0,1.0,98041.0,17.0,0.0,3,0,197012
1,1338267101,2005,74,240,5,0.0,2.0,1500.0,0.0,0.0,0,0,197012
2,1057042610,2011,75,3000,6,4.0,8.0,10500.0,13.0,0.0,3,0,197012
3,400121247,1982,55,1000,6,9.0,2.0,619318.0,31.0,0.0,6,0,197012
4,970179772,1986,53,150,17,4.0,1.0,13000.0,31.0,0.0,1,1,201612


In [649]:

baseinfo['EID'] = baseinfo['EID'].fillna(0)
baseinfo['CreateYear'] = baseinfo['CreateYear'].fillna(0)
baseinfo['TradeType'] = baseinfo['TradeType'].fillna(0).astype('int')
baseinfo['RegisteredCapital'] = baseinfo['RegisteredCapital'].fillna(0).astype('int')
baseinfo['Feature1'] = baseinfo['Feature1'].fillna(0)
baseinfo['Feature2'] = baseinfo['Feature2'].fillna(0)
baseinfo['Feature3'] = baseinfo['Feature3'].fillna(0)
baseinfo['Feature4'] = baseinfo['Feature4'].fillna(0)
baseinfo['Feature5'] = baseinfo['Feature5'].fillna(0)
train_labels['EndDate'] = train_labels['EndDate'].fillna('197012').astype('int')

merged_train = pd.merge(baseinfo,invest_data,on='EID',how='left')
merged_train = pd.merge(merged_train, train_labels, on='EID', how='right')
merged_train['counts'] = merged_train['counts'].fillna(0).astype('int')



print(len(merged_train))

merged_train.head()


150095


,EID,CreateYear,TradeType,RegisteredCapital,Type,Feature1,Feature2,Feature3,Feature4,Feature5,counts,Y,EndDate
0,732046267,1980,51,480,13,25.0,1.0,98041.0,17.0,0.0,3,0,197012
1,1338267101,2005,74,240,5,0.0,2.0,1500.0,0.0,0.0,0,0,197012
2,1057042610,2011,75,3000,6,4.0,8.0,10500.0,13.0,0.0,3,0,197012
3,400121247,1982,55,1000,6,9.0,2.0,619318.0,31.0,0.0,6,0,197012
4,970179772,1986,53,150,17,4.0,1.0,13000.0,31.0,0.0,1,1,201612


In [650]:
merged_test = pd.merge(baseinfo, invest_data, on='EID', how='left')
merged_test = pd.merge(merged_test, test_data, on='EID', how='right')
merged_test['counts'] = merged_test['counts'].fillna(0).astype('int')

test_x = merged_test.iloc[:,1:11]


test_x.head()



,CreateYear,TradeType,RegisteredCapital,Type,Feature1,Feature2,Feature3,Feature4,Feature5,counts
0,1993,74,30,13,0.0,1.0,570.0,3.0,0.0,0
1,2015,69,1000,6,2.0,1.0,1000.0,0.0,0.0,0
2,1995,54,14200,5,0.0,2.0,11280.0,5.0,0.0,3
3,2002,48,2800,7,0.0,22.0,100.0,0.0,0.0,1
4,2015,51,7000,6,3.0,2.0,909980.0,12.0,0.0,0


In [651]:
## split data to train and test
count = len(merged_train)


all_x = merged_train.iloc[:, 1:11]
all_x = all_x.values

all_y = merged_train.iloc[:, 11:12]
all_y = all_y.values


In [652]:
print(type(all_y),type(all_x))

all_x = sequence.pad_sequences(all_x)
all_y = sequence.pad_sequences(all_y)

print(all_x.shape)
print(all_y.shape)


<class 'numpy.ndarray'> <class 'numpy.ndarray'>


(150095, 10)
(150095, 1)


In [676]:
model = Sequential()

model.add(Dense(10,input_dim = 10))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


# 二分类问题
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])



model.fit(all_x, all_y,
          validation_split = 0.1,
          batch_size=128,
          epochs=8,
          )

model.evaluate(all_x, all_y)

Train on 135085 samples, validate on 15010 samples
Epoch 1/8


   128/135085 [..............................] - ETA: 1:06:16 - loss: 9.0905 - acc: 0.4062

  1536/135085 [..............................] - ETA: 5:32 - loss: 8.7243 - acc: 0.4264   

  3584/135085 [..............................] - ETA: 2:22 - loss: 8.5489 - acc: 0.4389

  5632/135085 [>.............................] - ETA: 1:30 - loss: 8.3450 - acc: 0.4513

...

147648/150095 [============================>.] - ETA: 0s

148544/150095 [============================>.] - ETA: 0s

149440/150095 [============================>.] - ETA: 0s

150095/150095 [==============================] - 8s 56us/step


[0.4036154936563127, 0.8643592391497306]

In [677]:
res = model.predict(test_x)

res


array([[0.13589258],
       [0.13203117],
       [0.07547836],
       ...,
       [0.13695367],
       [0.1367912 ],
       [0.13706934]], dtype=float32)

In [678]:
print(res.max(),res.min(),res.mean(),res.std())

0.13747844 0.0 0.13312592 0.017910052


In [679]:
print(res.shape)

sub = pd.DataFrame({'EID':merged_test.EID,'FORTARGET':[0 if i < 0.13 else 1 for i in res],'PROB':res.flatten()})
sub.to_csv("result/result.csv", index=0, float_format='%.4f')

(149715, 1)
